# Data Exploration - Dec 14, 2017 

In [49]:
# Import modules

import pandas as pd
import numpy as np
import os
import sys
import math
import pickle
import random
import re
import json
import datetime

import sqlite3

from dateutil.parser import parse

import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
#import colorlover as cl
from IPython.display import Image

src_dir = os.path.join(os.getcwd(), os.pardir,'src')
sys.path.append(src_dir)

import load_data as ld

init_notebook_mode(connected=True)

%reload_ext autoreload
%autoreload 2

In [7]:
data_dir = 'C:\\Users\\Colleen\\Documents\\Kaggle Corporacion\\raw_data' #'D:\Kaggle Data\Corporacion'
db_loc = data_dir
db_name = 'kaggle_corporacion_db'
db_f_name = os.path.join(data_dir, ld.DB_NAME)

## Data stats

In [ ]:
# Get number of rows in train.csv

f = open(os.path.join(data_dir, 'train.csv', 'r'))
row_count = sum(1 for row in f)
f.close()

There are 125497041 rows in train.csv

In [30]:
# Get number of items

connex = sqlite3.connect(os.path.join(db_loc, db_name + '.db'))
df = pd.read_sql_query("SELECT DISTINCT item_nbr FROM train_data;", connex)
connex.close()
print df.shape[0]

4036


There are 4036 items

In [12]:
# Get total time

connex = sqlite3.connect(os.path.join(db_loc, db_name + '.db'))

df = pd.read_sql_query("SELECT date FROM train_data ORDER BY ROWID ASC LIMIT 1", connex)
d_start = df['date'].iloc[0]

df = pd.read_sql_query("SELECT date FROM train_data ORDER BY ROWID DESC LIMIT 1", connex)
d_end = df['date'].iloc[0]

connex.close()

In [28]:
dst = parse(d_start)
dend = parse(d_end)

print dend - dst
print 'about ' + str(float(1687)/365.0) + ' years'

1687 days, 0:00:00
about 4.62191780822 years


In [46]:
import time
import calendar
import datetime

print d_start
calendar.timegm(time.strptime(d_start, '%Y-%m-%d %H:%M:%S'))


(datetime.datetime.strptime(d_end, '%Y-%m-%d %H:%M:%S') - datetime.datetime(2013,1,1)).total_seconds()

2013-01-01 00:00:00


145756800.0

In [42]:
print datetime.datetime(d_start)

TypeError: an integer is required

In [32]:
# Get total number of stores

connex = sqlite3.connect(os.path.join(db_loc, db_name + '.db'))
df = pd.read_sql_query("SELECT DISTINCT store_nbr FROM train_data;", connex)
connex.close()
print 'There are ' + str(df.shape[0]) + ' num stores'

There are 54 num stores


## Plots

In [16]:
# Load data from 2013
train_data_2013 = ld.get_year_data(db_loc, 2013)

### Store vs sales

These plots show differences between stores in sales of a couple items for a single year.  

Some stores appear to not sell any of the items in 2013 whereas some stores only sell the items a portion of the year and others sell the items nearly every day of the year.

The amount sold varies widely from store to store.  Its possible that some stores just see higher volume in general and thus show high amounts for every them.  So we can normalize by number of transactions.

In [19]:
# Get 3 random items

random.seed(3)
rand_items = random.sample(train_data_2013['item_nbr'], 3)


[1102971, 716245, 830797]


In [42]:
data = train_data_2013.loc[train_data_2013['item_nbr'] == rand_items[0],:]
grpd_data = data.groupby('store_nbr')
iplot([go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.count()['id'], name = 'num_trans'),
      go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.sum()['unit_sales'], name = 'num_days_in_year_item_sold')])

In [43]:
data = train_data_2013.loc[train_data_2013['item_nbr'] == rand_items[1],:]
grpd_data = data.groupby('store_nbr')
iplot([go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.count()['id'], name = 'num_trans'),
      go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.sum()['unit_sales'], name = 'num_days_in_year_item_sold')])

### Item vs sales

These plots show differences between in sales of all items for a couple stores.

There appears to be a couple items where both stores have good sales for the year, and also other items where only one store has sales. In addition, the second store appears to have more sales overall throughout the year than the first store. Its possible that the second store is in a more populated area or services more people than the first.

In [38]:
# Get 3 random stores

random.seed(3)
rand_stores = random.sample(train_data_2013['store_nbr'], 3)

In [44]:
data = train_data_2013.loc[train_data_2013['store_nbr'] == rand_stores[0],:]
grpd_data = data.groupby('item_nbr')
iplot(go.Figure(data = [go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.sum()['unit_sales'], name = 'num_trans')],
               layout = go.Layout(yaxis = dict(range = [0, 40000]))))

In [45]:
data = train_data_2013.loc[train_data_2013['store_nbr'] == rand_stores[1],:]
grpd_data = data.groupby('item_nbr')
iplot(go.Figure(data = [go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.sum()['unit_sales'], name = 'num_trans')],
               layout = go.Layout(yaxis = dict(range = [0, 40000]))))

### Item/Store vs Day of the Week

Here we look at how the day of the week affects sales of an item from a store by plotting the average sales for each day of the week throughout the year.

In the first store/item combination we see a increase in sales on Friday which could be due to shoppers hitting this store for this item before the weekend or possible due to paydays or holidays.

In [52]:
data = train_data_2013.loc[(train_data_2013['store_nbr'] == rand_stores[0]) & (train_data_2013['item_nbr'] == rand_items[0]),:]
week_day = [datetime.datetime(row['date_year'], row['date_month'], row['date_day']).weekday() for _,row in data.iterrows()]
data['week_day'] = week_day 
grpd_data = data.groupby('week_day')
iplot(go.Figure(data = [go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.mean()['unit_sales'], name = 'tot_trans')]))

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Here we look at the store's day of the week average overall items.  Here we see the sales are in general higher on Fridays.

In [53]:
data = train_data_2013.loc[train_data_2013['store_nbr'] == rand_stores[0],:]
week_day = [datetime.datetime(row['date_year'], row['date_month'], row['date_day']).weekday() for _,row in data.iterrows()]
data['week_day'] = week_day 
grpd_data = data.groupby('week_day')
iplot(go.Figure(data = [go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.mean()['unit_sales'], name = 'tot_trans')]))

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



### Item/Store vs Month

Here we look at the same store/item combination from above but over each month of the year.  We can see a spike around February and June.  Its possible that these are holiday months and the item is associated with the holidays.

In [54]:
data = train_data_2013.loc[(train_data_2013['store_nbr'] == rand_stores[0]) & (train_data_2013['item_nbr'] == rand_items[0]),:]
grpd_data = data.groupby('date_month')
iplot(go.Figure(data = [go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.sum()['unit_sales'], name = 'tot_trans')]))

Here we look at the same store sales from month to month over all items. We can see there's a spike in December, likely due to Christmas and a dip in January, likely due to fewer funds available after Christmas.  There trends don't quite match those above.  Thus the particular item from above has its own sales trend throughout the year.

In [55]:
data = train_data_2013.loc[train_data_2013['store_nbr'] == rand_stores[0],:]
grpd_data = data.groupby('date_month')
iplot(go.Figure(data = [go.Bar(x = [val for val,_ in grpd_data], y = grpd_data.sum()['unit_sales'], name = 'tot_trans')]))

Things that affect sales of an item:

-seasonality (is it in season)
-is it on promotion
-is it a holiday item x is it a holiday
-sales of similar items
-sales of items it is commonly bought with
-day of the week (bought more during common shopping days)
-oil prices (current price and current rate of change)
-is is a luxury item vs a staple
-is the store in a wealthy or poor area


Possible features:
    
-day of the week (1-7)
-is payday
-oil price
-short term oil trend
-long term oil trend
-price
-is on promotion
-is in season (alg to determine seasonality)
-how close to payday
-store cluster
-store itself
-

Things to plot/analyse

-location vs sales for a couple items
-day of the week vs sales for a couple items
-price of oil vs sales